In [1]:
import usb.core
import usb.util
import time

# Find your device
dev = usb.core.find(idVendor=0x359F, idProduct=0x3003)

# Check if the device was found
if dev is None:
    dev = usb.core.find(idVendor=0x359F, idProduct=0x3021)
    if dev is None:
        raise ValueError('Device not found')

# Print the device speed
speed_dict = {
    1: "Low Speed",  # USB 1.0
    2: "Full Speed", # USB 1.1
    3: "High Speed", # USB 2.0
    4: "Super Speed" # USB 3.0
}

print(f'Device speed: {speed_dict.get(dev.speed, "Unknown Speed")}')

# Set the configuration
dev.set_configuration()

# Get an endpoint instance
cfg = dev.get_active_configuration()
intf = cfg[(0, 0)]

# print(intf)

ep_out = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT and e.bEndpointAddress == 0x02
)

ep_in = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_IN and e.bEndpointAddress == 0x82
)

if ep_out is None or ep_in is None:
    raise ValueError('Endpoint not found')

## START 
print('\n\n')
## REG 4
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 4
wIndex = 0
data = [0x01, 0x00, 0x00, 0x00]

data_hex = [hex(x) for x in data]
print('REG 4 Write data:', data_hex)

# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)

print('REG 4 Write Response:', response1)


# Write data to the device
## data_to_write = b'\x01'
## ep_out.write(data_to_write)
## 

# Read data back from the device
start_time = time.time()
data_read_all = []
while True:
    data_read = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize)
    data_read_all.extend(data_read)
    if time.time() - start_time >= 2:
        break

# while data_read_all[0] != 0xaa:
#     data_read_all = data_read_all[1:]
for i in range(len(data_read_all[:256])//16):
    print(f'Data:', bytes(data_read_all[i*16:i*16+16]))

## STOP
print('\n\n')
## REG 4
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 4
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]

data_hex = [hex(x) for x in data]
print('REG 4 Write data:', data_hex)

# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)

print('REG 4 Write Response:', response1)

# Release the device resources
usb.util.dispose_resources(dev)

Device speed: High Speed



REG 4 Write data: ['0x1', '0x0', '0x0', '0x0']
REG 4 Write Response: 4
Data: b'\xaa\xcc\xf0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\x00\xff\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\xcc\xf0\xff\xff\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Data: b'\xaa\

In [ ]:
# data_read = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize)
# print('Data read:', data_read.tobytes())

usb.util.dispose_resources(dev)